In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

from scipy import sparse
from IPython.display import Image

from mpl_toolkits.mplot3d import Axes3D

sns.set(style="ticks", color_codes=True)
%matplotlib inline

import os
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from datetime import datetime
import dateutil

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

#For re-sizing
from skimage.transform import resize
from scipy import misc
import matplotlib.pyplot as plt
import skimage

In [3]:
paths = ["alexachung", "bucketlistjourney", "mariekondo", "jamieoliver", "gypsea_lust", 
         "kimkardashian", "tombrady", "ocasio2018"]
# filelist = os.listdir(path)
followers = {}
followers["alexachung"] = float(3300000)
followers["bucketlistjourney"] = float(100000)
followers["mariekondo"] = float(3000000)
followers["jamieoliver"] = float(6700000)
followers["gypsea_lust"] = float(2100000)
followers["kimkardashian"] = float(138000000)
followers["tombrady"] = float(6100000)
followers["ocasio2018"] = float(3400000)

In [4]:
image_arrays = []
for p in paths:
    filelist = os.listdir(p)
    for file in filelist:
        if str(file)[-1] != "g":
            continue
        pic = load_img(p + str("/") + str(file))
        arr = img_to_array(pic)
        image_arrays.append((p, str(file), arr))

In [5]:
dataDict = {}
for path in paths:
    with open(path + str("/") + path + str(".json")) as f:
        data = json.load(f)
    dataDict[path] = data

likes = {}
final_arr = []
likes_array = []
for img in image_arrays:
    jsonData = dataDict[img[0]]
    for i in range(len(jsonData['GraphImages'])):
        if img[1] in jsonData['GraphImages'][i]["display_url"]:
            likes[img[1]] = jsonData['GraphImages'][i]["edge_media_preview_like"]["count"]/followers[img[0]]
            final_arr.append(img)
            likes_array.append(jsonData['GraphImages'][i]["edge_media_preview_like"]["count"]/followers[img[0]])
            

print(len(image_arrays))
print(len(likes))

print(len(final_arr))
print(len(likes))

142
81
81
81


In [6]:
### Create datasets 
img_values = []
i_height = 32
i_width = 32
for img in final_arr:
    image = np.asarray(img[2])
    image = np.divide(image, float(255))
    image = skimage.transform.resize(image, (i_height, i_width), anti_aliasing=True, mode='reflect')
    img_values.append(image.flatten())

In [7]:
# Make y discrete

# y = np.asarray(likes)
y = np.asarray(likes_array)
print(type(y))
new_y = np.divide(y, 0.003)
print(y)
new_y = new_y.astype(int)
print(new_y)

<class 'numpy.ndarray'>
[0.00726273 0.00525697 0.00325788 0.01992364 0.00349242 0.00330152
 0.02612818 0.01230455 0.00345818 0.01527    0.01893    0.02101
 0.01268    0.01648    0.01537    0.01586    0.0143     0.01516
 0.005652   0.09890533 0.00689167 0.02318033 0.01667667 0.00846433
 0.018406   0.013795   0.011268   0.02625333 0.00457478 0.00309254
 0.00440985 0.00433552 0.00270791 0.00363343 0.02360238 0.02661571
 0.02395476 0.02778381 0.01838286 0.03593381 0.02072476 0.02280381
 0.02300667 0.03447238 0.01263476 0.01695762 0.00181449 0.01279488
 0.02663065 0.01418512 0.0121562  0.02043403 0.02202417 0.0157633
 0.01075553 0.04449575 0.02532577 0.03876755 0.00961025 0.01409393
 0.0625359  0.11832869 0.05896213 0.09238492 0.08708574 0.03607459
 0.05351656 0.05183902 0.0807677  0.06024459 0.07291098 0.06169623
 0.07888574 0.06452618 0.14062147 0.21582735 0.08647588 0.09412882
 0.03146059 0.06845324 0.24071853]
[ 2  1  1  6  1  1  8  4  1  5  6  7  4  5  5  5  4  5  1 32  2  7  5  2
  6 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(img_values, new_y, test_size=0.1, random_state=0)

k_dict = {}
k_choices = [1,2,3,5,7,10,12]
for k in k_choices:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    k_dict[k] = knn.score(X_test, y_test)

print(k_dict)

{1: 0.1111111111111111, 2: 0.1111111111111111, 3: 0.1111111111111111, 5: 0.1111111111111111, 7: 0.1111111111111111, 10: 0.1111111111111111, 12: 0.1111111111111111}


In [16]:
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'